# Ted Ed Riddle: Big Bang

[Source](https://www.youtube.com/watch?v=SEDabz-hyKo)

[Glitched Failure Video]()


## Proposed Algorithm:
- Search for optimum position by iterating over all positions
- Can ignore being to left of any antimatter particle
- ALWAYS better to be leftmost of any group of matter particles
- We need to position ourselves to the left of any last surviving matter particle before all are annihilated

## `Particle` class

- Will act as a graph node in a circular graph
- Uses a "doubly linked list"-style approach keeping track of `Particle` to it's left and right

In [68]:
class Particle:
    MATTER = '+'
    ANNIHILATED = '💀'
    ANTI_MATTER = '-'
    
    def __init__(self, particle_type, pos):
        self.left = None
        self.right = None
        self.to_be_annihilated = False
        self.state = particle_type
        self.pos = pos
        
    def annihilate(self):
        self.state = self.ANNIHILATED
        self.to_be_annihilated = False

## `Circle` class
- Will act as a wrapper class to manage `Particle`s
- Responsible for handling each "round" of annihilation.

In [89]:
class Circle:
    def __init__(self, particles):
        self.root = self.init_particles(particles)
        self.matter_count = len(particles) // 2 # half are matter particles

    def init_particles(self, particles):
        root = None
        prev_particle = None
        for i, particle_type in enumerate(particles):
            particle = Particle(particle_type=particle_type, pos=i)
            if root == None:
                root = particle
            else:
                prev_particle.left = particle
                particle.right = prev_particle
            prev_particle = particle
        
        # link last particle to root
        prev_particle.left = root
        root.right = prev_particle
        
        return root

    def print_circle(self):
        particle_states = []
        considered = set()
        curr_particle = self.root
        while curr_particle.pos not in considered:
            considered.add(curr_particle.pos)
            if curr_particle.state != Particle.ANNIHILATED:
                particle_states.append(f'{curr_particle.state}/{curr_particle.pos}')
            curr_particle = curr_particle.right
        print('..., ' + ', '.join(particle_states) + ', ...')
    
    def annihilate(self):
        """
        Go through all surviving particles and annihilate each pair where
        antimatter is to the right of a matter particle.
        """
        to_be_annihilated_list = []
        curr_particle = self.root

        while not curr_particle.to_be_annihilated:
            # only consider matter particles, skip over anything else
            # NOTE: moving to right always
            if curr_particle.state != Particle.MATTER:
                curr_particle = curr_particle.right
                continue
            # curr_particle.state is Particle.MATTER
            # secure "left" matter particle
            left_matter_particle = curr_particle
            curr_particle = curr_particle.right
            
            # skip over any ANNIHILATED
            while curr_particle.state == Particle.ANNIHILATED:
                curr_particle = curr_particle.right
            
            # Flag for annihilation
            if curr_particle.state == Particle.ANTI_MATTER:
                right_antimatter_particle = curr_particle
                left_matter_particle.to_be_annihilated = True
                right_antimatter_particle.to_be_annihilated = True
                to_be_annihilated_list.append(left_matter_particle)
                to_be_annihilated_list.append(right_antimatter_particle)
                curr_particle = curr_particle.right
        
        # Annihilate!
        for particle in to_be_annihilated_list:
            if particle.state == Particle.MATTER:
                self.matter_count -= 1
            particle.annihilate()
    
    def get_last_survivor_pos(self, verbose = False):
        # go through successive rounds of annihilation until last survivor(s)
        survivor_pos = set()
        i = 0
        while self.matter_count:
            if verbose:
                print(f'Round: {i}')
                i += 1
                self.print_circle()
                print('===========')
            self.annihilate()
            
            # record survivor positions
            curr_particle = self.root
            past_first = False
            temp_survivor_pos = set()
            while not (curr_particle.pos == 0 and past_first):
                if curr_particle.state == Particle.MATTER:
                    temp_survivor_pos.add(curr_particle.pos)
                curr_particle = curr_particle.right
                past_first = True
            if len(temp_survivor_pos):
                survivor_pos = temp_survivor_pos
        return survivor_pos

## 1st Circle

<img src='assets/circles.png' width='600px'>

In [90]:
particles_1 = [
    Particle.MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
]

In [91]:
circle_1 = Circle(particles_1)

In [92]:
circle_1.print_circle()

..., +/0, -/9, -/8, -/7, -/6, +/5, +/4, -/3, +/2, +/1, ...


In [93]:
circle_1.get_last_survivor_pos(verbose=True)

Round: 0
..., +/0, -/9, -/8, -/7, -/6, +/5, +/4, -/3, +/2, +/1, ...
Round: 1
..., -/8, -/7, -/6, +/5, +/2, +/1, ...
Round: 2
..., -/7, -/6, +/5, +/2, ...
Round: 3
..., -/6, +/5, ...


{5}

### Intrepretation:
- Being at leftmost on the largest group of consecutive matter particles is best 

## 2nd Circle

<img src='assets/circles.png' width='600px'>

In [94]:
particles_2 = [
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
]

In [95]:
circle_2 = Circle(particles_2)
circle_2.get_last_survivor_pos(verbose=True)

Round: 0
..., -/0, +/19, -/18, +/17, +/16, -/15, -/14, +/13, -/12, +/11, +/10, -/9, -/8, +/7, +/6, +/5, -/4, +/3, -/2, -/1, ...
Round: 1
..., -/0, +/17, -/14, +/11, -/8, +/7, +/6, -/1, ...
Round: 2
..., -/0, +/7, ...


{7}

### Intrepretation:
- Being at leftmost on the largest group of consecutive matter particles is best 

## 3rd Circle

<img src='assets/circles.png' width='600px'>

In [96]:
particles_3 = [
    Particle.MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.MATTER,
    Particle.MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
    Particle.ANTI_MATTER,
]

In [97]:
circle_3 = Circle(particles_3)
circle_3.get_last_survivor_pos(verbose=True)

Round: 0
..., +/0, -/23, -/22, -/21, -/20, -/19, +/18, +/17, -/16, +/15, -/14, -/13, -/12, +/11, +/10, +/9, -/8, +/7, -/6, -/5, +/4, +/3, +/2, +/1, ...
Round: 1
..., -/22, -/21, -/20, -/19, +/18, -/13, -/12, +/11, +/10, -/5, +/4, +/3, +/2, +/1, ...
Round: 2
..., -/21, -/20, -/19, -/12, +/11, +/4, +/3, +/2, ...
Round: 3
..., -/20, -/19, -/12, +/11, +/4, +/3, ...
Round: 4
..., -/19, -/12, +/11, +/4, ...
Round: 5
..., -/12, +/11, ...


{11}

### Intrepretation:
- Being at leftmost on the largest group of consecutive matter particles is best 

## Making our own examples!

`create_particles`:
- Takes `n` int argument
- Returns a list of `n` matter and antimatter particles randomly distributed 

In [98]:
import numpy as np

In [99]:
def create_particles(n = 10, seed = 123):
    np.random.seed(seed)
    PARTICLES = [Particle.MATTER, Particle.ANTI_MATTER]
    counts = {
        Particle.MATTER : n,
        Particle.ANTI_MATTER : n
    }
    
    output = []
    while len(output) < n * 2:
        total_p = sum(counts.values())
        p = [counts[particle_type] / total_p for particle_type in PARTICLES]
        choice = np.random.choice(PARTICLES, p = p)
        output.append(choice)
        counts[choice] -= 1
    return output

In [100]:
example_particles = create_particles(n=5, seed = 123)
example_particles

['-', '+', '+', '-', '-', '+', '-', '-', '+', '+']

In [101]:
from collections import Counter
occurrences = Counter(example_particles)
occurrences

Counter({'-': 5, '+': 5})

In [102]:
def print_val_with_index(arr):
    for i, val in enumerate(arr):
        print(f"{i}: {val}")

In [103]:
print_val_with_index(example_particles)

0: -
1: +
2: +
3: -
4: -
5: +
6: -
7: -
8: +
9: +


## Testing on larger custom examples

### Custom test #1

In [104]:
test_particles_1 = create_particles(n=15, seed=2)
print_val_with_index(test_particles_1)

0: +
1: +
2: -
3: +
4: +
5: +
6: +
7: -
8: +
9: +
10: -
11: -
12: +
13: -
14: +
15: -
16: -
17: -
18: -
19: +
20: -
21: +
22: -
23: +
24: +
25: -
26: +
27: -
28: -
29: -


In [105]:
test_circle_1 = Circle(test_particles_1)
test_circle_1.get_last_survivor_pos(verbose=True)

Round: 0
..., +/0, -/29, -/28, -/27, +/26, -/25, +/24, +/23, -/22, +/21, -/20, +/19, -/18, -/17, -/16, -/15, +/14, -/13, +/12, -/11, -/10, +/9, +/8, -/7, +/6, +/5, +/4, +/3, -/2, +/1, ...
Round: 1
..., -/28, -/27, +/24, -/17, -/16, -/15, -/10, +/9, +/6, +/5, +/4, +/1, ...
Round: 2
..., -/27, -/16, -/15, -/10, +/9, +/6, +/5, +/4, ...
Round: 3
..., -/16, -/15, -/10, +/9, +/6, +/5, ...
Round: 4
..., -/15, -/10, +/9, +/6, ...
Round: 5
..., -/10, +/9, ...


{9}

### Intrepretation:
- Being at leftmost on the largest group of consecutive matter particles is best 
- BUT we must consider "additive" effects from matter "clumping" together after consecutive rounds of annihilation

### Custom test #2

In [106]:
test_particles_2 = create_particles(n=30, seed=2)
print_val_with_index(test_particles_2)

0: +
1: +
2: -
3: +
4: +
5: +
6: +
7: -
8: +
9: +
10: -
11: -
12: +
13: -
14: +
15: -
16: -
17: -
18: -
19: +
20: -
21: +
22: +
23: +
24: +
25: -
26: +
27: +
28: +
29: +
30: -
31: +
32: -
33: -
34: -
35: +
36: -
37: -
38: +
39: -
40: -
41: -
42: -
43: +
44: -
45: +
46: -
47: -
48: -
49: -
50: +
51: +
52: +
53: -
54: +
55: +
56: +
57: -
58: -
59: -


In [86]:
test_circle_2 = Circle(test_particles_2)
test_circle_2.get_last_survivor_pos(verbose=True)

Round: 0
..., +/0, +/1, -/2, +/3, +/4, +/5, +/6, -/7, +/8, +/9, -/10, -/11, +/12, -/13, +/14, -/15, -/16, -/17, -/18, +/19, -/20, +/21, +/22, +/23, +/24, -/25, +/26, +/27, +/28, +/29, -/30, +/31, -/32, -/33, -/34, +/35, -/36, -/37, +/38, -/39, -/40, -/41, -/42, +/43, -/44, +/45, -/46, -/47, -/48, -/49, +/50, +/51, +/52, -/53, +/54, +/55, +/56, -/57, -/58, -/59, ...
Round: 1
..., +/0, +/3, +/4, +/5, +/8, -/11, -/16, -/17, -/18, +/21, +/22, +/23, +/26, +/27, +/28, -/33, -/34, -/37, -/40, -/41, -/42, -/47, -/48, -/49, +/50, +/51, +/54, +/55, -/58, -/59, ...
Round: 2
..., +/0, +/3, +/4, +/5, -/16, -/17, -/18, +/21, +/22, +/23, +/26, +/27, -/34, -/37, -/40, -/41, -/42, -/47, -/48, -/49, +/50, +/51, +/54, -/59, ...
Round: 3
..., +/0, +/3, +/4, -/17, -/18, +/21, +/22, +/23, +/26, -/37, -/40, -/41, -/42, -/47, -/48, -/49, +/50, +/51, ...
Round: 4
..., +/0, +/3, -/18, +/21, +/22, +/23, -/40, -/41, -/42, -/47, -/48, -/49, +/50, +/51, ...
Round: 5
..., +/0, +/21, +/22, -/41, -/42, -/47, -/48, -/4

{50}

### Intrepretation:
- Being at leftmost on the largest group of consecutive matter particles is best 
- BUT we must consider "additive" effects from matter "clumping" together after consecutive rounds of annihilation
- We can calculate this "additive" difference by taking a consecutive difference with matter particles adding 1 and antimatter particles subtracting 1. The best place to be will be to the right of the most negative difference.

### Intrepretation:
- Being at leftmost on the largest group of consecutive matter particles is best 
- BUT we must consider "additive" effects from matter "clumping" together after consecutive rounds of annihilation

In [107]:
# consecutive diff
diffs = []
for particle_type in test_particles_2:
    val = 1 if particle_type == Particle.MATTER else -1
    last_val = diffs[-1] if len(diffs) else 0
    diffs.append(last_val + val)

In [108]:
print_val_with_index(diffs)

0: 1
1: 2
2: 1
3: 2
4: 3
5: 4
6: 5
7: 4
8: 5
9: 6
10: 5
11: 4
12: 5
13: 4
14: 5
15: 4
16: 3
17: 2
18: 1
19: 2
20: 1
21: 2
22: 3
23: 4
24: 5
25: 4
26: 5
27: 6
28: 7
29: 8
30: 7
31: 8
32: 7
33: 6
34: 5
35: 6
36: 5
37: 4
38: 5
39: 4
40: 3
41: 2
42: 1
43: 2
44: 1
45: 2
46: 1
47: 0
48: -1
49: -2
50: -1
51: 0
52: 1
53: 0
54: 1
55: 2
56: 3
57: 2
58: 1
59: 0


### Intrepretation:
- By taking this running consecutive sum, we can deduce the spot we can survive at
- The lowest point occurs at the index where there is the longest run of antimatter particles
- Therefore, the spot just to the right of that index is the best spot to be!


<img src='assets/circles.png' width='600px'>